In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.compose import ColumnTransformer
from termcolor import colored
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer



from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline

# Data Pura

In [37]:
data = pd.read_csv('./data_identificada.csv')
data.head(1)

,index,Name,Netflows,First_Protocol,Second_Protocol,Third_Protocol,p1_d,p2_d,p3_d,duration,...,second_dp,third_dp,p1_ip,p2_ip,p3_ip,p1_ib,p2_ib,p3_ib,Type,Cryptocurrency
0,2169,1113-csves/1.csv,400,TCP,UDP,NaN,0.0,0.0,9.04175,4287.992,...,53.0,80.0,1.0,1.0,5.5,32.0,33.0,873.0,not_mine,Does not have


In [38]:
data = data.drop(['Second_Protocol', 'Third_Protocol'], axis = 1)

## Eliminar caracteristicas que no aportan valor

In [39]:
data = data.drop(['index', 'Name'], axis = 1)

## Imputación de valores ausentes

In [40]:
punter = pd.concat([data['second_sp'],data['second_dp'],data['third_sp'],data['third_dp']], axis = 1)
imputer = SimpleImputer(missing_values = np.nan, strategy = "median")
values = imputer.fit_transform(punter)

In [41]:
temp = data.drop(['second_sp','second_dp','third_sp','third_dp'], axis = 1)
punter = pd.DataFrame(values, columns = punter.columns)
data = pd.concat([temp,punter], axis = 1)
data.head(2)

,Netflows,First_Protocol,p1_d,p2_d,p3_d,duration,max_d,min_d,#packets,Avg_bps,...,p3_ip,p1_ib,p2_ib,p3_ib,Type,Cryptocurrency,second_sp,second_dp,third_sp,third_dp
0,400,TCP,0.0,0.0,9.04175,4287.992,65.519,0.0,6750,360512,...,5.5,32.0,33.0,873.00,not_mine,Does not have,53.0,53.0,80.0,80.0
1,46,UDP,0.0,0.0,0.00000,0.000,0.000,0.0,46,1296,...,1.0,33.0,49.0,110.75,not_mine,Does not have,49129.0,49129.0,47733.0,47733.0


# Exclusión de variables con varianza próxima a cero

# Eliminar la multicolinialidad

## OneHotEncoder

In [42]:
data_categoric = data[data.select_dtypes(include=['object']).columns.to_list()]
one_hot =  OneHotEncoder(drop="first")
one_hot.fit_transform(data_categoric)

<4733x9 sparse matrix of type '<class 'numpy.float64'>'
	with 11481 stored elements in Compressed Sparse Row format>

In [43]:
one_hot.categories_

[array(['ICMP', 'ICMP6', 'TCP', 'UDP'], dtype=object),
 array(['mine', 'not_mine'], dtype=object),
 array(['Bitcash', 'Bitcoin', 'Does not have', 'Etherium', 'Litecoin',
        'Monero'], dtype=object)]

In [44]:
dataDummy = pd.get_dummies(data_categoric)
dataDummy.head()

,First_Protocol_ICMP,First_Protocol_ICMP6,First_Protocol_TCP,First_Protocol_UDP,Type_mine,Type_not_mine,Cryptocurrency_Bitcash,Cryptocurrency_Bitcoin,Cryptocurrency_Does not have,Cryptocurrency_Etherium,Cryptocurrency_Litecoin,Cryptocurrency_Monero
0,0,0,1,0,0,1,0,0,1,0,0,0
1,0,0,0,1,0,1,0,0,1,0,0,0
2,0,0,1,0,0,1,0,0,1,0,0,0
3,0,0,1,0,1,0,0,1,0,0,0,0
4,0,0,1,0,0,1,0,0,1,0,0,0


## Estandarización

In [45]:
data_numeric = data[data.select_dtypes(include=['float64', 'int64']).columns.to_list()]

In [46]:
preprocessor = ColumnTransformer([
    ('scale', StandardScaler(), data_numeric.columns), 
], remainder='passthrough')

In [47]:
values = preprocessor.fit_transform(data_numeric)
values

array([[ 1.5753182 , -0.70094869, -0.80447685, ..., -0.80602925,
        -0.48227582, -0.36157031],
       [-0.27272335, -0.70094869, -0.80447685, ...,  1.11078473,
         2.01069298,  2.94298939],
       [ 0.47380191, -0.15963957, -0.24099573, ..., -0.80329519,
        -0.48227582, -0.36157031],
       ...,
       [-0.50242342,  0.70358428,  0.64043234, ...,  1.3982912 ,
        -0.48002628, -0.3363976 ],
       [-0.50242342,  2.11540637,  2.09284154, ..., -0.67776254,
        -0.48002628, -0.3363976 ],
       [-0.50242342,  2.43640092,  2.42316607, ...,  1.48773419,
        -0.48002628, -0.3363976 ]])

In [48]:
data_standarizada =  pd.DataFrame(values, columns = data_numeric.columns)
data_standarizada.head(1)

,Netflows,p1_d,p2_d,p3_d,duration,max_d,min_d,#packets,Avg_bps,Avg_pps,...,p1_ip,p2_ip,p3_ip,p1_ib,p2_ib,p3_ib,second_sp,second_dp,third_sp,third_dp
0,1.575318,-0.700949,-0.804477,-0.909496,0.054551,-0.864733,-0.625504,0.570941,1.236305,1.285332,...,-0.368592,-0.401682,-0.402089,-0.424797,-0.488919,-0.510645,-0.825878,-0.806029,-0.482276,-0.36157


### Concatenación de los conjuntos de datos


In [49]:
data_p = pd.concat([data_standarizada, dataDummy], axis = 1)
data_p.columns

Index(['Netflows', 'p1_d', 'p2_d', 'p3_d', 'duration', 'max_d', 'min_d',
       '#packets', 'Avg_bps', 'Avg_pps', 'Avg_bpp', '#Bytes', '#sp', '#dp',
       'first_sp', 'first_dp', 'p1_ip', 'p2_ip', 'p3_ip', 'p1_ib', 'p2_ib',
       'p3_ib', 'second_sp', 'second_dp', 'third_sp', 'third_dp',
       'First_Protocol_ICMP', 'First_Protocol_ICMP6', 'First_Protocol_TCP',
       'First_Protocol_UDP', 'Type_mine', 'Type_not_mine',
       'Cryptocurrency_Bitcash', 'Cryptocurrency_Bitcoin',
       'Cryptocurrency_Does not have', 'Cryptocurrency_Etherium',
       'Cryptocurrency_Litecoin', 'Cryptocurrency_Monero'],
      dtype='object')

In [50]:
data_p.to_csv('dataA.csv', index=False)

# Data No Pura

In [51]:
data_n = pd.read_csv('./data_no_identificada.csv')
data_n.head(1)

,index,Name,Netflows,First_Protocol,Second_Protocol,Third_Protocol,p1_d,p2_d,p3_d,duration,...,second_dp,third_dp,p1_ip,p2_ip,p3_ip,p1_ib,p2_ib,p3_ib,Type,Cryptocurrency
0,36016,20-csves/8177.csv,17,UDP,ICMP6,NaN,0.0,3.001,360.024,2790.015,...,1900.0,17500.0,1.0,2.0,4.0,32.0,500.0,700.0,mine,Unidentified


In [52]:
data_n = data_n.drop(['Second_Protocol','Third_Protocol'], axis = 1)

## Eliminar caracteristicas que no aportan valor

In [53]:
data_n = data_n.drop(['index','Name'], axis = 1)
data_n.columns

Index(['Netflows', 'First_Protocol', 'p1_d', 'p2_d', 'p3_d', 'duration',
       'max_d', 'min_d', '#packets', 'Avg_bps', 'Avg_pps', 'Avg_bpp', '#Bytes',
       '#sp', '#dp', 'first_sp', 'second_sp', 'third_sp', 'first_dp',
       'second_dp', 'third_dp', 'p1_ip', 'p2_ip', 'p3_ip', 'p1_ib', 'p2_ib',
       'p3_ib', 'Type', 'Cryptocurrency'],
      dtype='object')

## Identificar valores nulos

In [54]:
data_n.isnull().sum()

Netflows              0
First_Protocol        0
p1_d                  0
p2_d                  0
p3_d                  0
duration              0
max_d                 0
min_d                 0
#packets              0
Avg_bps               0
Avg_pps               0
Avg_bpp               0
#Bytes                0
#sp                   0
#dp                   0
first_sp              0
second_sp          7498
third_sp          10993
first_dp              0
second_dp          1708
third_dp           7103
p1_ip                 0
p2_ip                 0
p3_ip                 0
p1_ib                 0
p2_ib                 0
p3_ib                 0
Type                  0
Cryptocurrency        0
dtype: int64

## Imputar valores

In [55]:
punter = pd.concat([data_n['second_sp'],data_n['third_sp'],data_n['second_dp'],data_n['third_dp']], axis = 1)
imputer = SimpleImputer(missing_values = np.nan, strategy = 'median')
values = imputer.fit_transform(punter)

In [56]:
values = pd.DataFrame(values, columns = punter.columns)

In [57]:
data_n['second_sp'] = values['second_sp']
data_n['third_sp'] = values['third_sp']
data_n['second_dp'] = values['second_dp']
data_n['third_dp'] = values['third_dp']

## OneHotEncoder

In [58]:
data_categoric = data_n.select_dtypes(['object'])
data_categoric.columns

Index(['First_Protocol', 'Type', 'Cryptocurrency'], dtype='object')

In [59]:
one_hot =  OneHotEncoder()
one_hot.fit_transform(data_categoric)

<39554x8 sparse matrix of type '<class 'numpy.float64'>'
	with 118662 stored elements in Compressed Sparse Row format>

In [60]:
dataDummy = pd.get_dummies(data_categoric)
dataDummy.head()

,First_Protocol_ICMP,First_Protocol_ICMP6,First_Protocol_TCP,First_Protocol_UDP,Type_mine,Type_not_mine,Cryptocurrency_Does not have,Cryptocurrency_Unidentified
0,0,0,0,1,1,0,0,1
1,0,1,0,0,1,0,0,1
2,0,1,0,0,1,0,0,1
3,0,1,0,0,1,0,0,1
4,0,1,0,0,1,0,0,1


## Estandarización

In [ ]:
data_numeric = data_n.select_dtypes(['int64','float64'])

In [ ]:
preprocessor = ColumnTransformer([
    ('scale', StandardScaler(), data_numeric.columns), 
], remainder='passthrough')

In [ ]:
values = preprocessor.fit_transform(data_numeric)

In [ ]:
data_estandarizada = pd.DataFrame(values,columns= data_numeric.columns)
data_estandarizada.head(1)

### Concatenación de los conjuntos de datos

In [ ]:
data_n = pd.concat([data_estandarizada,dataDummy],axis = 1)
data_n.head()

In [ ]:
data_n.to_csv('dataB.csv', index = False)

https://machinelearningmastery.com/iterative-imputation-for-missing-values-in-machine-learning/